In [1]:
import talib
import os
import numpy as np
import pandas as pd
import pickle
import quandl
import datetime
import plotly.offline as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
py.init_notebook_mode(connected=True)
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten,Convolution2D,MaxPooling2D,Activation,Reshape,Permute
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

from keras.layers import LSTM,Bidirectional

Using TensorFlow backend.


In [2]:
finalprice=pd.read_csv('fianaldata.csv',index_col='date')
finalprice.index=pd.to_datetime(finalprice.index,format="%Y-%m-%d")
finalprice=finalprice['2016-10-30':'2019-03-31']
finalprice.replace(0,np.nan,inplace=True)
finalprice=finalprice.fillna(method='pad')

In [3]:
price= finalprice.loc['2018-04-01':'2019-03-31',['Close']]
price=price.values
trueprice=finalprice.loc['2018-04-01':'2019-03-31',['predict']]
forecase=np.zeros((365,))

In [4]:
final=finalprice['2017-10-30':'2018-04-30']

data=final.values
def get_train_data(batch_size=1,time_step=3,train_begin=0,train_end=(len(final)-30)):
    batch_index=[]
    data_train=data[train_begin:train_end]
    mean=np.mean(data_train,axis=0)
    std=np.std(data_train,axis=0)
    normalized_train_data=(data_train-mean)/std
    size=(len(normalized_train_data)+time_step-1)//time_step
    train_x,train_y=[],[]
    for i in range(len(normalized_train_data)-time_step+1):
        if i % batch_size==0:
            batch_index.append(i)
        x=normalized_train_data[i:i+time_step,:17]
        #y=normalized_train_data[i:i+time_step,17,np.newaxis]
        y=normalized_train_data[i+time_step-1,17,np.newaxis]
        train_x.append(x.tolist())
        train_y.append(y.tolist())
    return mean,std,train_x,train_y
mean,std,train_x,train_y=get_train_data()
def get_test_data(time_step=3,test_begin=(len(final)-30-2)):
    data_test=data[test_begin:]
    normalized_test_data=(data_test-mean)/std
   # size=(len(normalized_test_data)+time_step-1)//time_step
    size=(len(normalized_test_data)-time_step+1)
    test_x,test_y=[],[]
    for i in range(size):
        #x=normalized_test_data[i*time_step:(i+1)*time_step,:17]
        #y=normalized_test_data[i*time_step:(i+1)*time_step,17]
        x=normalized_test_data[i:i+time_step,:17]
        y=normalized_test_data[i+time_step-1,17,np.newaxis]
        test_x.append(x.tolist())
        test_y.append(y.tolist())
    return test_x,test_y
test_x,test_y=get_test_data()
train_x=np.array(train_x)
train_x=train_x.reshape(train_x.shape[0],3,17,1)
train_y=np.array(train_y)
train_y=train_y.reshape((len(final)-2-30),1)
test_x=np.array(test_x)
test_x=test_x.reshape(30,3,17,1)
test_y=np.array(test_y)
test_y=test_y.reshape(30,1)

model = Sequential()
model.add(Convolution2D(
    input_shape=(3,
                 17,
                1),
   kernel_size=(2,2),
    filters=15,
    padding='same'
))

model.add(Activation('relu'))
model.add(MaxPooling2D(
    pool_size=(2,2),
    strides=(2,2),
    padding='same'))
#model.add(Dropout(0.2))

model.add(Convolution2D(
    input_shape=(3,
                 17,
                 1),
    kernel_size=(2,2),
    filters=15,
    padding='same'
))



model.add(Convolution2D(
    input_shape=(3,
                 17,
                 1),
    kernel_size=(2,2),
    filters=15,
    padding='same'
))


model.add(Flatten())
model.add(Dense(25))
model.add(Activation('relu'))


#model.add(Dense(15))


model.add(Reshape((1,25),input_shape=(25,)))
#model.add(Permute(1,15))
#model = Sequential()
model.add(LSTM(50,return_sequences=False))

#model.add(LSTM(30, input_shape=(1, 50),return_sequences=False))
#model.add(Dropout(0.3))
#model.add(LSTM(15,return_sequences=True))

#model.add(LSTM(10,return_sequences=False))
#model.add(Dropout(0.2))

model.add(Dense(1))

model.add(Dropout(0.1))

adam=Adam(lr=0.01)
model.compile(loss='mse', optimizer='adam')
history = model.fit(train_x, train_y, epochs=100, batch_size=50, validation_data=(test_x, test_y), verbose=0,shuffle=False)
    
test_y=np.array(test_y)
yhat = model.predict(test_x)
Yhat=yhat*std[17]+mean[17]
Test_y=test_y*std[17]+mean[17]
Test_y=Test_y.reshape(30,)
Yhat=np.array(Yhat)
Yhat=Yhat.reshape(30,)

for i in range(365-365,395-365): 
    forecase[i]=Yhat[i-365+365]    


In [5]:
final=finalprice['2017-11-29':'2018-05-31']

data=final.values
def get_train_data(batch_size=1,time_step=3,train_begin=0,train_end=(len(final)-31)):
    batch_index=[]
    data_train=data[train_begin:train_end]
    mean=np.mean(data_train,axis=0)
    std=np.std(data_train,axis=0)
    normalized_train_data=(data_train-mean)/std
    size=(len(normalized_train_data)+time_step-1)//time_step
    train_x,train_y=[],[]
    for i in range(len(normalized_train_data)-time_step+1):
        if i % batch_size==0:
            batch_index.append(i)
        x=normalized_train_data[i:i+time_step,:17]
        #y=normalized_train_data[i:i+time_step,17,np.newaxis]
        y=normalized_train_data[i+time_step-1,17,np.newaxis]
        train_x.append(x.tolist())
        train_y.append(y.tolist())
    return mean,std,train_x,train_y
mean,std,train_x,train_y=get_train_data()
def get_test_data(time_step=3,test_begin=(len(final)-31-2)):
    data_test=data[test_begin:]
    normalized_test_data=(data_test-mean)/std
   # size=(len(normalized_test_data)+time_step-1)//time_step
    size=(len(normalized_test_data)-time_step+1)
    test_x,test_y=[],[]
    for i in range(size):
        #x=normalized_test_data[i*time_step:(i+1)*time_step,:17]
        #y=normalized_test_data[i*time_step:(i+1)*time_step,17]
        x=normalized_test_data[i:i+time_step,:17]
        y=normalized_test_data[i+time_step-1,17,np.newaxis]
        test_x.append(x.tolist())
        test_y.append(y.tolist())
    return test_x,test_y
test_x,test_y=get_test_data()
train_x=np.array(train_x)
train_x=train_x.reshape(train_x.shape[0],3,17,1)
train_y=np.array(train_y)
train_y=train_y.reshape((len(final)-2-31),1)
test_x=np.array(test_x)
test_x=test_x.reshape(31,3,17,1)
test_y=np.array(test_y)
test_y=test_y.reshape(31,1)

model = Sequential()
model.add(Convolution2D(
    input_shape=(3,
                 17,
                1),
   kernel_size=(2,2),
    filters=15,
    padding='same'
))

model.add(Activation('relu'))
model.add(MaxPooling2D(
    pool_size=(2,2),
    strides=(2,2),
    padding='same'))
#model.add(Dropout(0.2))

model.add(Convolution2D(
    input_shape=(3,
                 17,
                 1),
    kernel_size=(2,2),
    filters=15,
    padding='same'
))



model.add(Convolution2D(
    input_shape=(3,
                 17,
                 1),
    kernel_size=(2,2),
    filters=15,
    padding='same'
))


model.add(Flatten())
model.add(Dense(25))
model.add(Activation('relu'))


#model.add(Dense(15))


model.add(Reshape((1,25),input_shape=(25,)))
#model.add(Permute(1,15))
#model = Sequential()
model.add(LSTM(50,return_sequences=False))

#model.add(LSTM(30, input_shape=(1, 50),return_sequences=False))
#model.add(Dropout(0.3))
#model.add(LSTM(15,return_sequences=True))

#model.add(LSTM(10,return_sequences=False))
#model.add(Dropout(0.2))

model.add(Dense(1))

model.add(Dropout(0.1))

adam=Adam(lr=0.01)
model.compile(loss='mse', optimizer='adam')
history = model.fit(train_x, train_y, epochs=100, batch_size=50, validation_data=(test_x, test_y), verbose=0,shuffle=False)
    
test_y=np.array(test_y)
yhat = model.predict(test_x)
Yhat=yhat*std[17]+mean[17]
Test_y=test_y*std[17]+mean[17]
Test_y=Test_y.reshape(31,)
Yhat=np.array(Yhat)
Yhat=Yhat.reshape(31,)

for i in range(395-365,426-365): 
    forecase[i]=Yhat[i-395+365]    
forecase

In [6]:
final=finalprice['2017-12-30':'2018-06-30']

data=final.values
def get_train_data(batch_size=1,time_step=3,train_begin=0,train_end=(len(final)-30)):
    batch_index=[]
    data_train=data[train_begin:train_end]
    mean=np.mean(data_train,axis=0)
    std=np.std(data_train,axis=0)
    normalized_train_data=(data_train-mean)/std
    size=(len(normalized_train_data)+time_step-1)//time_step
    train_x,train_y=[],[]
    for i in range(len(normalized_train_data)-time_step+1):
        if i % batch_size==0:
            batch_index.append(i)
        x=normalized_train_data[i:i+time_step,:17]
        #y=normalized_train_data[i:i+time_step,17,np.newaxis]
        y=normalized_train_data[i+time_step-1,17,np.newaxis]
        train_x.append(x.tolist())
        train_y.append(y.tolist())
    return mean,std,train_x,train_y
mean,std,train_x,train_y=get_train_data()
def get_test_data(time_step=3,test_begin=(len(final)-30-2)):
    data_test=data[test_begin:]
    normalized_test_data=(data_test-mean)/std
   # size=(len(normalized_test_data)+time_step-1)//time_step
    size=(len(normalized_test_data)-time_step+1)
    test_x,test_y=[],[]
    for i in range(size):
        #x=normalized_test_data[i*time_step:(i+1)*time_step,:17]
        #y=normalized_test_data[i*time_step:(i+1)*time_step,17]
        x=normalized_test_data[i:i+time_step,:17]
        y=normalized_test_data[i+time_step-1,17,np.newaxis]
        test_x.append(x.tolist())
        test_y.append(y.tolist())
    return test_x,test_y
test_x,test_y=get_test_data()
train_x=np.array(train_x)
train_x=train_x.reshape(train_x.shape[0],3,17,1)
train_y=np.array(train_y)
train_y=train_y.reshape((len(final)-2-30),1)
test_x=np.array(test_x)
test_x=test_x.reshape(30,3,17,1)
test_y=np.array(test_y)
test_y=test_y.reshape(30,1)

model = Sequential()
model.add(Convolution2D(
    input_shape=(3,
                 17,
                1),
   kernel_size=(2,2),
    filters=15,
    padding='same'
))

model.add(Activation('relu'))
model.add(MaxPooling2D(
    pool_size=(2,2),
    strides=(2,2),
    padding='same'))
#model.add(Dropout(0.2))

model.add(Convolution2D(
    input_shape=(3,
                 17,
                 1),
    kernel_size=(2,2),
    filters=15,
    padding='same'
))



model.add(Convolution2D(
    input_shape=(3,
                 17,
                 1),
    kernel_size=(2,2),
    filters=15,
    padding='same'
))


model.add(Flatten())
model.add(Dense(25))
model.add(Activation('relu'))


#model.add(Dense(15))


model.add(Reshape((1,25),input_shape=(25,)))
#model.add(Permute(1,15))
#model = Sequential()
model.add(LSTM(50,return_sequences=False))

#model.add(LSTM(30, input_shape=(1, 50),return_sequences=False))
#model.add(Dropout(0.3))
#model.add(LSTM(15,return_sequences=True))

#model.add(LSTM(10,return_sequences=False))
#model.add(Dropout(0.2))

model.add(Dense(1))

model.add(Dropout(0.1))

adam=Adam(lr=0.01)
model.compile(loss='mse', optimizer='adam')
history = model.fit(train_x, train_y, epochs=100, batch_size=50, validation_data=(test_x, test_y), verbose=0,shuffle=False)
    
test_y=np.array(test_y)
yhat = model.predict(test_x)
Yhat=yhat*std[17]+mean[17]
Test_y=test_y*std[17]+mean[17]
Test_y=Test_y.reshape(30,)
Yhat=np.array(Yhat)
Yhat=Yhat.reshape(30,)

for i in range(426-365,456-365): 
    forecase[i]=Yhat[i-426+365]    


In [7]:
final=finalprice['2018-01-30':'2018-07-31']

data=final.values
def get_train_data(batch_size=1,time_step=3,train_begin=0,train_end=(len(final)-31)):
    batch_index=[]
    data_train=data[train_begin:train_end]
    mean=np.mean(data_train,axis=0)
    std=np.std(data_train,axis=0)
    normalized_train_data=(data_train-mean)/std
    size=(len(normalized_train_data)+time_step-1)//time_step
    train_x,train_y=[],[]
    for i in range(len(normalized_train_data)-time_step+1):
        if i % batch_size==0:
            batch_index.append(i)
        x=normalized_train_data[i:i+time_step,:17]
        #y=normalized_train_data[i:i+time_step,17,np.newaxis]
        y=normalized_train_data[i+time_step-1,17,np.newaxis]
        train_x.append(x.tolist())
        train_y.append(y.tolist())
    return mean,std,train_x,train_y
mean,std,train_x,train_y=get_train_data()
def get_test_data(time_step=3,test_begin=(len(final)-31-2)):
    data_test=data[test_begin:]
    normalized_test_data=(data_test-mean)/std
   # size=(len(normalized_test_data)+time_step-1)//time_step
    size=(len(normalized_test_data)-time_step+1)
    test_x,test_y=[],[]
    for i in range(size):
        #x=normalized_test_data[i*time_step:(i+1)*time_step,:17]
        #y=normalized_test_data[i*time_step:(i+1)*time_step,17]
        x=normalized_test_data[i:i+time_step,:17]
        y=normalized_test_data[i+time_step-1,17,np.newaxis]
        test_x.append(x.tolist())
        test_y.append(y.tolist())
    return test_x,test_y
test_x,test_y=get_test_data()
train_x=np.array(train_x)
train_x=train_x.reshape(train_x.shape[0],3,17,1)
train_y=np.array(train_y)
train_y=train_y.reshape((len(final)-2-31),1)
test_x=np.array(test_x)
test_x=test_x.reshape(31,3,17,1)
test_y=np.array(test_y)
test_y=test_y.reshape(31,1)

model = Sequential()
model.add(Convolution2D(
    input_shape=(3,
                 17,
                1),
   kernel_size=(2,2),
    filters=15,
    padding='same'
))

model.add(Activation('relu'))
model.add(MaxPooling2D(
    pool_size=(2,2),
    strides=(2,2),
    padding='same'))
#model.add(Dropout(0.2))

model.add(Convolution2D(
    input_shape=(3,
                 17,
                 1),
    kernel_size=(2,2),
    filters=15,
    padding='same'
))



model.add(Convolution2D(
    input_shape=(3,
                 17,
                 1),
    kernel_size=(2,2),
    filters=15,
    padding='same'
))


model.add(Flatten())
model.add(Dense(25))
model.add(Activation('relu'))


#model.add(Dense(15))


model.add(Reshape((1,25),input_shape=(25,)))
#model.add(Permute(1,15))
#model = Sequential()
model.add(LSTM(50,return_sequences=False))

#model.add(LSTM(30, input_shape=(1, 50),return_sequences=False))
#model.add(Dropout(0.3))
#model.add(LSTM(15,return_sequences=True))

#model.add(LSTM(10,return_sequences=False))
#model.add(Dropout(0.2))

model.add(Dense(1))

model.add(Dropout(0.1))

adam=Adam(lr=0.01)
model.compile(loss='mse', optimizer='adam')
history = model.fit(train_x, train_y, epochs=100, batch_size=50, validation_data=(test_x, test_y), verbose=0,shuffle=False)
    
test_y=np.array(test_y)
yhat = model.predict(test_x)
Yhat=yhat*std[17]+mean[17]
Test_y=test_y*std[17]+mean[17]
Test_y=Test_y.reshape(31,)
Yhat=np.array(Yhat)
Yhat=Yhat.reshape(31,)

for i in range(456-365,487-365): 
    forecase[i]=Yhat[i-456+365]    


In [8]:
final=finalprice['2018-02-27':'2018-08-31']

data=final.values
def get_train_data(batch_size=1,time_step=3,train_begin=0,train_end=(len(final)-31)):
    batch_index=[]
    data_train=data[train_begin:train_end]
    mean=np.mean(data_train,axis=0)
    std=np.std(data_train,axis=0)
    normalized_train_data=(data_train-mean)/std
    size=(len(normalized_train_data)+time_step-1)//time_step
    train_x,train_y=[],[]
    for i in range(len(normalized_train_data)-time_step+1):
        if i % batch_size==0:
            batch_index.append(i)
        x=normalized_train_data[i:i+time_step,:17]
        #y=normalized_train_data[i:i+time_step,17,np.newaxis]
        y=normalized_train_data[i+time_step-1,17,np.newaxis]
        train_x.append(x.tolist())
        train_y.append(y.tolist())
    return mean,std,train_x,train_y
mean,std,train_x,train_y=get_train_data()
def get_test_data(time_step=3,test_begin=(len(final)-31-2)):
    data_test=data[test_begin:]
    normalized_test_data=(data_test-mean)/std
   # size=(len(normalized_test_data)+time_step-1)//time_step
    size=(len(normalized_test_data)-time_step+1)
    test_x,test_y=[],[]
    for i in range(size):
        #x=normalized_test_data[i*time_step:(i+1)*time_step,:17]
        #y=normalized_test_data[i*time_step:(i+1)*time_step,17]
        x=normalized_test_data[i:i+time_step,:17]
        y=normalized_test_data[i+time_step-1,17,np.newaxis]
        test_x.append(x.tolist())
        test_y.append(y.tolist())
    return test_x,test_y
test_x,test_y=get_test_data()
train_x=np.array(train_x)
train_x=train_x.reshape(train_x.shape[0],3,17,1)
train_y=np.array(train_y)
train_y=train_y.reshape((len(final)-2-31),1)
test_x=np.array(test_x)
test_x=test_x.reshape(31,3,17,1)
test_y=np.array(test_y)
test_y=test_y.reshape(31,1)

model = Sequential()
model.add(Convolution2D(
    input_shape=(3,
                 17,
                1),
   kernel_size=(2,2),
    filters=15,
    padding='same'
))

model.add(Activation('relu'))
model.add(MaxPooling2D(
    pool_size=(2,2),
    strides=(2,2),
    padding='same'))
#model.add(Dropout(0.2))

model.add(Convolution2D(
    input_shape=(3,
                 17,
                 1),
    kernel_size=(2,2),
    filters=15,
    padding='same'
))



model.add(Convolution2D(
    input_shape=(3,
                 17,
                 1),
    kernel_size=(2,2),
    filters=15,
    padding='same'
))


model.add(Flatten())
model.add(Dense(25))
model.add(Activation('relu'))


#model.add(Dense(15))


model.add(Reshape((1,25),input_shape=(25,)))
#model.add(Permute(1,15))
#model = Sequential()
model.add(LSTM(50,return_sequences=False))

#model.add(LSTM(30, input_shape=(1, 50),return_sequences=False))
#model.add(Dropout(0.3))
#model.add(LSTM(15,return_sequences=True))

#model.add(LSTM(10,return_sequences=False))
#model.add(Dropout(0.2))

model.add(Dense(1))

model.add(Dropout(0.1))

adam=Adam(lr=0.01)
model.compile(loss='mse', optimizer='adam')
history = model.fit(train_x, train_y, epochs=100, batch_size=50, validation_data=(test_x, test_y), verbose=0,shuffle=False)
    
test_y=np.array(test_y)
yhat = model.predict(test_x)
Yhat=yhat*std[17]+mean[17]
Test_y=test_y*std[17]+mean[17]
Test_y=Test_y.reshape(31,)
Yhat=np.array(Yhat)
Yhat=Yhat.reshape(31,)

for i in range(487-365,518-365): 
    forecase[i]=Yhat[i-487+365]    


In [9]:
final=finalprice['2018-03-30':'2018-09-30']

data=final.values
def get_train_data(batch_size=1,time_step=3,train_begin=0,train_end=(len(final)-30)):
    batch_index=[]
    data_train=data[train_begin:train_end]
    mean=np.mean(data_train,axis=0)
    std=np.std(data_train,axis=0)
    normalized_train_data=(data_train-mean)/std
    size=(len(normalized_train_data)+time_step-1)//time_step
    train_x,train_y=[],[]
    for i in range(len(normalized_train_data)-time_step+1):
        if i % batch_size==0:
            batch_index.append(i)
        x=normalized_train_data[i:i+time_step,:17]
        #y=normalized_train_data[i:i+time_step,17,np.newaxis]
        y=normalized_train_data[i+time_step-1,17,np.newaxis]
        train_x.append(x.tolist())
        train_y.append(y.tolist())
    return mean,std,train_x,train_y
mean,std,train_x,train_y=get_train_data()
def get_test_data(time_step=3,test_begin=(len(final)-30-2)):
    data_test=data[test_begin:]
    normalized_test_data=(data_test-mean)/std
   # size=(len(normalized_test_data)+time_step-1)//time_step
    size=(len(normalized_test_data)-time_step+1)
    test_x,test_y=[],[]
    for i in range(size):
        #x=normalized_test_data[i*time_step:(i+1)*time_step,:17]
        #y=normalized_test_data[i*time_step:(i+1)*time_step,17]
        x=normalized_test_data[i:i+time_step,:17]
        y=normalized_test_data[i+time_step-1,17,np.newaxis]
        test_x.append(x.tolist())
        test_y.append(y.tolist())
    return test_x,test_y
test_x,test_y=get_test_data()
train_x=np.array(train_x)
train_x=train_x.reshape(train_x.shape[0],3,17,1)
train_y=np.array(train_y)
train_y=train_y.reshape((len(final)-2-30),1)
test_x=np.array(test_x)
test_x=test_x.reshape(30,3,17,1)
test_y=np.array(test_y)
test_y=test_y.reshape(30,1)

model = Sequential()
model.add(Convolution2D(
    input_shape=(3,
                 17,
                1),
   kernel_size=(2,2),
    filters=15,
    padding='same'
))

model.add(Activation('relu'))
model.add(MaxPooling2D(
    pool_size=(2,2),
    strides=(2,2),
    padding='same'))
#model.add(Dropout(0.2))

model.add(Convolution2D(
    input_shape=(3,
                 17,
                 1),
    kernel_size=(2,2),
    filters=15,
    padding='same'
))



model.add(Convolution2D(
    input_shape=(3,
                 17,
                 1),
    kernel_size=(2,2),
    filters=15,
    padding='same'
))


model.add(Flatten())
model.add(Dense(25))
model.add(Activation('relu'))


#model.add(Dense(15))


model.add(Reshape((1,25),input_shape=(25,)))
#model.add(Permute(1,15))
#model = Sequential()
model.add(LSTM(50,return_sequences=False))

#model.add(LSTM(30, input_shape=(1, 50),return_sequences=False))
#model.add(Dropout(0.3))
#model.add(LSTM(15,return_sequences=True))

#model.add(LSTM(10,return_sequences=False))
#model.add(Dropout(0.2))

model.add(Dense(1))

model.add(Dropout(0.1))

adam=Adam(lr=0.01)
model.compile(loss='mse', optimizer='adam')
history = model.fit(train_x, train_y, epochs=100, batch_size=50, validation_data=(test_x, test_y), verbose=0,shuffle=False)
    
test_y=np.array(test_y)
yhat = model.predict(test_x)
Yhat=yhat*std[17]+mean[17]
Test_y=test_y*std[17]+mean[17]
Test_y=Test_y.reshape(30,)
Yhat=np.array(Yhat)
Yhat=Yhat.reshape(30,)

for i in range(518-365,548-365): 
    forecase[i]=Yhat[i-518+365]    


In [10]:
final=finalprice['2018-04-29':'2018-10-31']

data=final.values
def get_train_data(batch_size=1,time_step=3,train_begin=0,train_end=(len(final)-31)):
    batch_index=[]
    data_train=data[train_begin:train_end]
    mean=np.mean(data_train,axis=0)
    std=np.std(data_train,axis=0)
    normalized_train_data=(data_train-mean)/std
    size=(len(normalized_train_data)+time_step-1)//time_step
    train_x,train_y=[],[]
    for i in range(len(normalized_train_data)-time_step+1):
        if i % batch_size==0:
            batch_index.append(i)
        x=normalized_train_data[i:i+time_step,:17]
        #y=normalized_train_data[i:i+time_step,17,np.newaxis]
        y=normalized_train_data[i+time_step-1,17,np.newaxis]
        train_x.append(x.tolist())
        train_y.append(y.tolist())
    return mean,std,train_x,train_y
mean,std,train_x,train_y=get_train_data()
def get_test_data(time_step=3,test_begin=(len(final)-31-2)):
    data_test=data[test_begin:]
    normalized_test_data=(data_test-mean)/std
   # size=(len(normalized_test_data)+time_step-1)//time_step
    size=(len(normalized_test_data)-time_step+1)
    test_x,test_y=[],[]
    for i in range(size):
        #x=normalized_test_data[i*time_step:(i+1)*time_step,:17]
        #y=normalized_test_data[i*time_step:(i+1)*time_step,17]
        x=normalized_test_data[i:i+time_step,:17]
        y=normalized_test_data[i+time_step-1,17,np.newaxis]
        test_x.append(x.tolist())
        test_y.append(y.tolist())
    return test_x,test_y
test_x,test_y=get_test_data()
train_x=np.array(train_x)
train_x=train_x.reshape(train_x.shape[0],3,17,1)
train_y=np.array(train_y)
train_y=train_y.reshape((len(final)-2-31),1)
test_x=np.array(test_x)
test_x=test_x.reshape(31,3,17,1)
test_y=np.array(test_y)
test_y=test_y.reshape(31,1)

model = Sequential()
model.add(Convolution2D(
    input_shape=(3,
                 17,
                1),
   kernel_size=(2,2),
    filters=15,
    padding='same'
))

model.add(Activation('relu'))
model.add(MaxPooling2D(
    pool_size=(2,2),
    strides=(2,2),
    padding='same'))
#model.add(Dropout(0.2))

model.add(Convolution2D(
    input_shape=(3,
                 17,
                 1),
    kernel_size=(2,2),
    filters=15,
    padding='same'
))



model.add(Convolution2D(
    input_shape=(3,
                 17,
                 1),
    kernel_size=(2,2),
    filters=15,
    padding='same'
))


model.add(Flatten())
model.add(Dense(25))
model.add(Activation('relu'))


#model.add(Dense(15))


model.add(Reshape((1,25),input_shape=(25,)))
#model.add(Permute(1,15))
#model = Sequential()
model.add(LSTM(50,return_sequences=False))

#model.add(LSTM(30, input_shape=(1, 50),return_sequences=False))
#model.add(Dropout(0.3))
#model.add(LSTM(15,return_sequences=True))

#model.add(LSTM(10,return_sequences=False))
#model.add(Dropout(0.2))

model.add(Dense(1))

model.add(Dropout(0.1))

adam=Adam(lr=0.01)
model.compile(loss='mse', optimizer='adam')
history = model.fit(train_x, train_y, epochs=100, batch_size=50, validation_data=(test_x, test_y), verbose=0,shuffle=False)
    
test_y=np.array(test_y)
yhat = model.predict(test_x)
Yhat=yhat*std[17]+mean[17]
Test_y=test_y*std[17]+mean[17]
Test_y=Test_y.reshape(31,)
Yhat=np.array(Yhat)
Yhat=Yhat.reshape(31,)

for i in range(548-365,579-365): 
    forecase[i]=Yhat[i-548+365]    
forecase

In [11]:
final=finalprice['2018-05-30':'2018-11-30']

data=final.values
def get_train_data(batch_size=1,time_step=3,train_begin=0,train_end=(len(final)-30)):
    batch_index=[]
    data_train=data[train_begin:train_end]
    mean=np.mean(data_train,axis=0)
    std=np.std(data_train,axis=0)
    normalized_train_data=(data_train-mean)/std
    size=(len(normalized_train_data)+time_step-1)//time_step
    train_x,train_y=[],[]
    for i in range(len(normalized_train_data)-time_step+1):
        if i % batch_size==0:
            batch_index.append(i)
        x=normalized_train_data[i:i+time_step,:17]
        #y=normalized_train_data[i:i+time_step,17,np.newaxis]
        y=normalized_train_data[i+time_step-1,17,np.newaxis]
        train_x.append(x.tolist())
        train_y.append(y.tolist())
    return mean,std,train_x,train_y
mean,std,train_x,train_y=get_train_data()
def get_test_data(time_step=3,test_begin=(len(final)-30-2)):
    data_test=data[test_begin:]
    normalized_test_data=(data_test-mean)/std
   # size=(len(normalized_test_data)+time_step-1)//time_step
    size=(len(normalized_test_data)-time_step+1)
    test_x,test_y=[],[]
    for i in range(size):
        #x=normalized_test_data[i*time_step:(i+1)*time_step,:17]
        #y=normalized_test_data[i*time_step:(i+1)*time_step,17]
        x=normalized_test_data[i:i+time_step,:17]
        y=normalized_test_data[i+time_step-1,17,np.newaxis]
        test_x.append(x.tolist())
        test_y.append(y.tolist())
    return test_x,test_y
test_x,test_y=get_test_data()
train_x=np.array(train_x)
train_x=train_x.reshape(train_x.shape[0],3,17,1)
train_y=np.array(train_y)
train_y=train_y.reshape((len(final)-2-30),1)
test_x=np.array(test_x)
test_x=test_x.reshape(30,3,17,1)
test_y=np.array(test_y)
test_y=test_y.reshape(30,1)

model = Sequential()
model.add(Convolution2D(
    input_shape=(3,
                 17,
                1),
   kernel_size=(2,2),
    filters=15,
    padding='same'
))

model.add(Activation('relu'))
model.add(MaxPooling2D(
    pool_size=(2,2),
    strides=(2,2),
    padding='same'))
#model.add(Dropout(0.2))

model.add(Convolution2D(
    input_shape=(3,
                 17,
                 1),
    kernel_size=(2,2),
    filters=15,
    padding='same'
))



model.add(Convolution2D(
    input_shape=(3,
                 17,
                 1),
    kernel_size=(2,2),
    filters=15,
    padding='same'
))


model.add(Flatten())
model.add(Dense(25))
model.add(Activation('relu'))


#model.add(Dense(15))


model.add(Reshape((1,25),input_shape=(25,)))
#model.add(Permute(1,15))
#model = Sequential()
model.add(LSTM(50,return_sequences=False))

#model.add(LSTM(30, input_shape=(1, 50),return_sequences=False))
#model.add(Dropout(0.3))
#model.add(LSTM(15,return_sequences=True))

#model.add(LSTM(10,return_sequences=False))
#model.add(Dropout(0.2))

model.add(Dense(1))

model.add(Dropout(0.1))

adam=Adam(lr=0.01)
model.compile(loss='mse', optimizer='adam')
history = model.fit(train_x, train_y, epochs=100, batch_size=50, validation_data=(test_x, test_y), verbose=0,shuffle=False)
    
test_y=np.array(test_y)
yhat = model.predict(test_x)
Yhat=yhat*std[17]+mean[17]
Test_y=test_y*std[17]+mean[17]
Test_y=Test_y.reshape(30,)
Yhat=np.array(Yhat)
Yhat=Yhat.reshape(30,)

for i in range(579-365,609-365): 
    forecase[i]=Yhat[i-579+365]    
forecase

In [12]:
final=finalprice['2018-06-29':'2018-12-31']

data=final.values
def get_train_data(batch_size=1,time_step=3,train_begin=0,train_end=(len(final)-31)):
    batch_index=[]
    data_train=data[train_begin:train_end]
    mean=np.mean(data_train,axis=0)
    std=np.std(data_train,axis=0)
    normalized_train_data=(data_train-mean)/std
    size=(len(normalized_train_data)+time_step-1)//time_step
    train_x,train_y=[],[]
    for i in range(len(normalized_train_data)-time_step+1):
        if i % batch_size==0:
            batch_index.append(i)
        x=normalized_train_data[i:i+time_step,:17]
        #y=normalized_train_data[i:i+time_step,17,np.newaxis]
        y=normalized_train_data[i+time_step-1,17,np.newaxis]
        train_x.append(x.tolist())
        train_y.append(y.tolist())
    return mean,std,train_x,train_y
mean,std,train_x,train_y=get_train_data()
def get_test_data(time_step=3,test_begin=(len(final)-31-2)):
    data_test=data[test_begin:]
    normalized_test_data=(data_test-mean)/std
   # size=(len(normalized_test_data)+time_step-1)//time_step
    size=(len(normalized_test_data)-time_step+1)
    test_x,test_y=[],[]
    for i in range(size):
        #x=normalized_test_data[i*time_step:(i+1)*time_step,:17]
        #y=normalized_test_data[i*time_step:(i+1)*time_step,17]
        x=normalized_test_data[i:i+time_step,:17]
        y=normalized_test_data[i+time_step-1,17,np.newaxis]
        test_x.append(x.tolist())
        test_y.append(y.tolist())
    return test_x,test_y
test_x,test_y=get_test_data()
train_x=np.array(train_x)
train_x=train_x.reshape(train_x.shape[0],3,17,1)
train_y=np.array(train_y)
train_y=train_y.reshape((len(final)-2-31),1)
test_x=np.array(test_x)
test_x=test_x.reshape(31,3,17,1)
test_y=np.array(test_y)
test_y=test_y.reshape(31,1)

model = Sequential()
model.add(Convolution2D(
    input_shape=(3,
                 17,
                1),
   kernel_size=(2,2),
    filters=15,
    padding='same'
))

model.add(Activation('relu'))
model.add(MaxPooling2D(
    pool_size=(2,2),
    strides=(2,2),
    padding='same'))
#model.add(Dropout(0.2))

model.add(Convolution2D(
    input_shape=(3,
                 17,
                 1),
    kernel_size=(2,2),
    filters=15,
    padding='same'
))



model.add(Convolution2D(
    input_shape=(3,
                 17,
                 1),
    kernel_size=(2,2),
    filters=15,
    padding='same'
))


model.add(Flatten())
model.add(Dense(25))
model.add(Activation('relu'))


#model.add(Dense(15))


model.add(Reshape((1,25),input_shape=(25,)))
#model.add(Permute(1,15))
#model = Sequential()
model.add(LSTM(50,return_sequences=False))

#model.add(LSTM(30, input_shape=(1, 50),return_sequences=False))
#model.add(Dropout(0.3))
#model.add(LSTM(15,return_sequences=True))

#model.add(LSTM(10,return_sequences=False))
#model.add(Dropout(0.2))

model.add(Dense(1))

model.add(Dropout(0.1))

adam=Adam(lr=0.01)
model.compile(loss='mse', optimizer='adam')
history = model.fit(train_x, train_y, epochs=100, batch_size=50, validation_data=(test_x, test_y), verbose=0,shuffle=False)
    
test_y=np.array(test_y)
yhat = model.predict(test_x)
Yhat=yhat*std[17]+mean[17]
Test_y=test_y*std[17]+mean[17]
Test_y=Test_y.reshape(31,)
Yhat=np.array(Yhat)
Yhat=Yhat.reshape(31,)

for i in range(609-365,640-365): 
    forecase[i]=Yhat[i-609+365]    


In [13]:
final=finalprice['2018-07-30':'2019-01-31']

data=final.values
def get_train_data(batch_size=1,time_step=3,train_begin=0,train_end=(len(final)-31)):
    batch_index=[]
    data_train=data[train_begin:train_end]
    mean=np.mean(data_train,axis=0)
    std=np.std(data_train,axis=0)
    normalized_train_data=(data_train-mean)/std
    size=(len(normalized_train_data)+time_step-1)//time_step
    train_x,train_y=[],[]
    for i in range(len(normalized_train_data)-time_step+1):
        if i % batch_size==0:
            batch_index.append(i)
        x=normalized_train_data[i:i+time_step,:17]
        #y=normalized_train_data[i:i+time_step,17,np.newaxis]
        y=normalized_train_data[i+time_step-1,17,np.newaxis]
        train_x.append(x.tolist())
        train_y.append(y.tolist())
    return mean,std,train_x,train_y
mean,std,train_x,train_y=get_train_data()
def get_test_data(time_step=3,test_begin=(len(final)-31-2)):
    data_test=data[test_begin:]
    normalized_test_data=(data_test-mean)/std
   # size=(len(normalized_test_data)+time_step-1)//time_step
    size=(len(normalized_test_data)-time_step+1)
    test_x,test_y=[],[]
    for i in range(size):
        #x=normalized_test_data[i*time_step:(i+1)*time_step,:17]
        #y=normalized_test_data[i*time_step:(i+1)*time_step,17]
        x=normalized_test_data[i:i+time_step,:17]
        y=normalized_test_data[i+time_step-1,17,np.newaxis]
        test_x.append(x.tolist())
        test_y.append(y.tolist())
    return test_x,test_y
test_x,test_y=get_test_data()
train_x=np.array(train_x)
train_x=train_x.reshape(train_x.shape[0],3,17,1)
train_y=np.array(train_y)
train_y=train_y.reshape((len(final)-2-31),1)
test_x=np.array(test_x)
test_x=test_x.reshape(31,3,17,1)
test_y=np.array(test_y)
test_y=test_y.reshape(31,1)

model = Sequential()
model.add(Convolution2D(
    input_shape=(3,
                 17,
                1),
   kernel_size=(2,2),
    filters=15,
    padding='same'
))

model.add(Activation('relu'))
model.add(MaxPooling2D(
    pool_size=(2,2),
    strides=(2,2),
    padding='same'))
#model.add(Dropout(0.2))

model.add(Convolution2D(
    input_shape=(3,
                 17,
                 1),
    kernel_size=(2,2),
    filters=15,
    padding='same'
))



model.add(Convolution2D(
    input_shape=(3,
                 17,
                 1),
    kernel_size=(2,2),
    filters=15,
    padding='same'
))


model.add(Flatten())
model.add(Dense(25))
model.add(Activation('relu'))


#model.add(Dense(15))


model.add(Reshape((1,25),input_shape=(25,)))
#model.add(Permute(1,15))
#model = Sequential()
model.add(LSTM(50,return_sequences=False))

#model.add(LSTM(30, input_shape=(1, 50),return_sequences=False))
#model.add(Dropout(0.3))
#model.add(LSTM(15,return_sequences=True))

#model.add(LSTM(10,return_sequences=False))
#model.add(Dropout(0.2))

model.add(Dense(1))

model.add(Dropout(0.1))

adam=Adam(lr=0.01)
model.compile(loss='mse', optimizer='adam')
history = model.fit(train_x, train_y, epochs=100, batch_size=50, validation_data=(test_x, test_y), verbose=0,shuffle=False)
    
test_y=np.array(test_y)
yhat = model.predict(test_x)
Yhat=yhat*std[17]+mean[17]
Test_y=test_y*std[17]+mean[17]
Test_y=Test_y.reshape(31,)
Yhat=np.array(Yhat)
Yhat=Yhat.reshape(31,)

for i in range(640-365,671-365): 
    forecase[i]=Yhat[i-640+365]    


In [14]:
final=finalprice['2018-08-30':'2019-02-28']

data=final.values
def get_train_data(batch_size=1,time_step=3,train_begin=0,train_end=(len(final)-28)):
    batch_index=[]
    data_train=data[train_begin:train_end]
    mean=np.mean(data_train,axis=0)
    std=np.std(data_train,axis=0)
    normalized_train_data=(data_train-mean)/std
    size=(len(normalized_train_data)+time_step-1)//time_step
    train_x,train_y=[],[]
    for i in range(len(normalized_train_data)-time_step+1):
        if i % batch_size==0:
            batch_index.append(i)
        x=normalized_train_data[i:i+time_step,:17]
        #y=normalized_train_data[i:i+time_step,17,np.newaxis]
        y=normalized_train_data[i+time_step-1,17,np.newaxis]
        train_x.append(x.tolist())
        train_y.append(y.tolist())
    return mean,std,train_x,train_y
mean,std,train_x,train_y=get_train_data()
def get_test_data(time_step=3,test_begin=(len(final)-28-2)):
    data_test=data[test_begin:]
    normalized_test_data=(data_test-mean)/std
   # size=(len(normalized_test_data)+time_step-1)//time_step
    size=(len(normalized_test_data)-time_step+1)
    test_x,test_y=[],[]
    for i in range(size):
        #x=normalized_test_data[i*time_step:(i+1)*time_step,:17]
        #y=normalized_test_data[i*time_step:(i+1)*time_step,17]
        x=normalized_test_data[i:i+time_step,:17]
        y=normalized_test_data[i+time_step-1,17,np.newaxis]
        test_x.append(x.tolist())
        test_y.append(y.tolist())
    return test_x,test_y
test_x,test_y=get_test_data()
train_x=np.array(train_x)
train_x=train_x.reshape(train_x.shape[0],3,17,1)
train_y=np.array(train_y)
train_y=train_y.reshape((len(final)-2-28),1)
test_x=np.array(test_x)
test_x=test_x.reshape(28,3,17,1)
test_y=np.array(test_y)
test_y=test_y.reshape(28,1)

model = Sequential()
model.add(Convolution2D(
    input_shape=(3,
                 17,
                1),
   kernel_size=(2,2),
    filters=15,
    padding='same'
))

model.add(Activation('relu'))
model.add(MaxPooling2D(
    pool_size=(2,2),
    strides=(2,2),
    padding='same'))
#model.add(Dropout(0.2))

model.add(Convolution2D(
    input_shape=(3,
                 17,
                 1),
    kernel_size=(2,2),
    filters=15,
    padding='same'
))



model.add(Convolution2D(
    input_shape=(3,
                 17,
                 1),
    kernel_size=(2,2),
    filters=15,
    padding='same'
))


model.add(Flatten())
model.add(Dense(25))
model.add(Activation('relu'))


#model.add(Dense(15))


model.add(Reshape((1,25),input_shape=(25,)))
#model.add(Permute(1,15))
#model = Sequential()
model.add(LSTM(50,return_sequences=False))

#model.add(LSTM(30, input_shape=(1, 50),return_sequences=False))
#model.add(Dropout(0.3))
#model.add(LSTM(15,return_sequences=True))

#model.add(LSTM(10,return_sequences=False))
#model.add(Dropout(0.2))

model.add(Dense(1))

model.add(Dropout(0.1))

adam=Adam(lr=0.01)
model.compile(loss='mse', optimizer='adam')
history = model.fit(train_x, train_y, epochs=100, batch_size=50, validation_data=(test_x, test_y), verbose=0,shuffle=False)
    
test_y=np.array(test_y)
yhat = model.predict(test_x)
Yhat=yhat*std[17]+mean[17]
Test_y=test_y*std[17]+mean[17]
Test_y=Test_y.reshape(28,)
Yhat=np.array(Yhat)
Yhat=Yhat.reshape(28,)

for i in range(671-365,699-365): 
    forecase[i]=Yhat[i-671+365]    


In [15]:
final=finalprice['2018-09-29':'2019-03-31']

data=final.values

def get_train_data(batch_size=1,time_step=3,train_begin=0,train_end=(len(final)-31)):
    batch_index=[]
    data_train=data[train_begin:train_end]
    mean=np.mean(data_train,axis=0)
    std=np.std(data_train,axis=0)
    normalized_train_data=(data_train-mean)/std
    size=(len(normalized_train_data)+time_step-1)//time_step
    train_x,train_y=[],[]
    for i in range(len(normalized_train_data)-time_step+1):
        if i % batch_size==0:
            batch_index.append(i)
        x=normalized_train_data[i:i+time_step,:17]
        y=normalized_train_data[i+time_step-1,17,np.newaxis]
        train_x.append(x.tolist())
        train_y.append(y.tolist())
    return mean,std,train_x,train_y
mean,std,train_x,train_y=get_train_data()
def get_test_data(time_step=3,test_begin=(len(final)-31-2)):
    data_test=data[test_begin:]
    normalized_test_data=(data_test-mean)/std
    size=(len(normalized_test_data)-time_step+1)
    test_x,test_y=[],[]
    for i in range(size):
        x=normalized_test_data[i:i+time_step,:17]
        y=normalized_test_data[i+time_step-1,17,np.newaxis]
        test_x.append(x.tolist())
        test_y.append(y.tolist())
    return test_x,test_y
test_x,test_y=get_test_data()
train_x=np.array(train_x)
train_x=train_x.reshape(train_x.shape[0],3,17,1)
train_y=np.array(train_y)
train_y=train_y.reshape((len(final)-2-31),1)
test_x=np.array(test_x)
test_x=test_x.reshape(31,3,17,1)
test_y=np.array(test_y)
test_y=test_y.reshape(31,1)

model = Sequential()
model.add(Convolution2D(
    input_shape=(3,
                 17,
                1),
   kernel_size=(2,2),
    filters=15,
    padding='same'
))
model.add(Activation('relu'))
model.add(MaxPooling2D(
    pool_size=(2,2),
    strides=(2,2),
    padding='same'))
model.add(Convolution2D(
    input_shape=(3,
                 17,
                 1),
    kernel_size=(2,2),
    filters=15,
    padding='same'
))
model.add(Convolution2D(
    input_shape=(3,
                 17,
                 1),
    kernel_size=(2,2),
    filters=15,
    padding='same'
))
model.add(Flatten())
model.add(Dense(25))
model.add(Activation('relu'))
model.add(Reshape((1,25),input_shape=(25,)))
model.add(LSTM(50,return_sequences=False))
model.add(Dense(1))
model.add(Dropout(0.1))
adam=Adam(lr=0.01)
model.compile(loss='mse', optimizer='adam')
history = model.fit(train_x, train_y, epochs=100, batch_size=50, validation_data=(test_x, test_y), verbose=0,shuffle=False)
    
test_y=np.array(test_y)
yhat = model.predict(test_x)
Yhat=yhat*std[17]+mean[17]
Test_y=test_y*std[17]+mean[17]
Test_y=Test_y.reshape(31,)
Yhat=np.array(Yhat)
Yhat=Yhat.reshape(31,)

for i in range(699-365,730-365): 
    forecase[i]=Yhat[i-699+365]    
forecase

In [16]:
from matplotlib import pyplot
%matplotlib inline
trueprice=trueprice.values

In [17]:

pyplot.plot(forecase)
pyplot.plot(trueprice)
pyplot.show

In [18]:
trueprice=trueprice.reshape(365,)
forecase=np.array(forecase)
forecase=forecase.reshape(365,)
error=abs(trueprice-forecase)
error1=error/trueprice
pyplot.plot(error1)
pyplot.show

In [19]:
error2=error*error 
rmse=pow(sum(error2)/365,0.5) 
mae=sum(error)/365
mape=100*sum(error/trueprice)/365
x=np.zeros((365,)) 
y=np.zeros((365,)) 
a=np.zeros((365,)) 
b=np.zeros((365,)) 
c=np.zeros((365,)) 
for i in range(365): 
    if trueprice[i]>=price[i]:
        x[i]=1 
    if forecase[i]>=price[i]: 
        y[i]=1 
for i in range(365): 
    if x[i]==1 and y[i]==1: 
        a[i]=1 
    if x[i]==1 and y[i]==0: 
        b[i]=1 
    if x[i]==0 and y[i]==1: 
        c[i]=1 
precision=sum(a)/(sum(a)+sum(b)) 
recall=sum(a)/(sum(a)+sum(c)) 
f1=2*precision*recall/(precision+recall) 
outcome=np.zeros((371,)) 
for i in range(365): 
    outcome[i]=forecase[i] 
outcome[365]=mae 
outcome[366]=rmse 
outcome[367]=mape
outcome[368]=precision 
outcome[369]=recall 
outcome[370]=f1 
outcome=pd.DataFrame(outcome) 
outcome

In [20]:

outcome.to_csv('data20.csv')

In [21]:
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

In [4]:
trueprice.to_csv('trueprice.csv')

In [5]:
trueprice

,predict
date,
2018-04-01,7054.7
2018-04-02,7414.8
2018-04-03,6790.4
2018-04-04,6763.5
2018-04-05,6616.2
2018-04-06,6893.1
2018-04-07,7014.5
2018-04-08,6773.4
2018-04-09,6830.0


In [6]:
finalprice

,Open,High,Low,Close,Volume (BTC),Volume (Currency),index,biaopu,exchange,gold,nasdaq,niujiaosuo,oil,rate,RSI,mfi,obv,predict
date,,,,,,,,,,,,,,,,,,
2016-10-30,713.949,713.989,688.000,700.917,1750.780129,1.228539e+06,5565,2126.41,6.7858,1273.00,5190.10,10476.62,48.70,0.41,65.293669,6.197086e+01,59041.91285,698.919
2016-10-31,700.079,708.899,685.700,698.919,982.631046,6.878437e+05,4467,2126.15,6.7641,1272.00,5189.14,10481.89,46.86,0.31,60.736373,3.790429e+01,58059.28180,733.744
2016-11-01,698.790,739.488,697.910,733.744,2336.219977,1.702318e+06,4587,2111.72,6.7734,1288.45,5153.58,10414.05,46.67,0.41,86.100549,4.691699e+01,60395.50178,747.070
2016-11-02,733.742,747.070,722.890,747.070,1902.393632,1.390995e+06,6337,2097.94,6.7562,1303.75,5105.57,10334.50,45.34,0.41,89.860276,8.187062e+01,62297.89541,692.179
2016-11-03,747.070,750.500,673.407,692.179,4169.139506,2.969056e+06,6845,2088.66,6.7491,1301.00,5058.41,10307.64,44.66,0.41,33.639305,5.129104e+01,58128.75590,706.036
2016-11-04,692.902,709.998,682.311,706.036,1904.910013,1.335331e+06,6811,2085.18,6.7514,1302.80,5046.37,10289.35,44.07,0.41,46.349772,2.475579e+01,60033.66592,706.478
2016-11-05,706.036,714.471,698.478,706.478,630.687729,4.452695e+05,6048,2085.18,6.7514,1302.80,5046.37,10289.35,44.07,0.41,46.836969,9.442540e+00,60664.35365,715.497
2016-11-06,706.460,721.458,702.879,715.497,638.744099,4.555510e+05,4572,2085.18,6.7514,1302.80,5046.37,10289.35,44.07,0.41,58.399626,4.034681e+01,61303.09775,707.345
2016-11-07,715.000,715.413,700.160,707.345,917.369981,6.484827e+05,5370,2131.52,6.7725,1283.05,5166.17,10500.16,44.89,0.41,45.100424,5.812730e+01,60385.72776,712.500
